# Генератор медицинской вопросно-ответной системы

В этом ноутбуке создадим медицинскую вопросно-ответную систему, обучив GPT на датасете https://huggingface.co/datasets/blinoff/medical_qa_ru_data.

Установим необходимое окружение:

In [1]:
!pip install -q transformers

     |████████████████████████████████| 5.3 MB 14.5 MB/s 
     |████████████████████████████████| 7.6 MB 54.2 MB/s 
     |████████████████████████████████| 163 kB 72.6 MB/s 


In [2]:
import numpy as np
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split

Примонтируем гугл-диск и скачаем файл с датасетом:

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
path =  "/content/drive/My Drive/GeekBrains/NLP/Course Project/"

In [5]:
data = pd.read_csv(path + 'qa_medical.csv')
data.dropna(inplace=True)
data

,desc,ans,class
0,"Здравствуйте, в июле после лечения антибиотика...",У вас возможны кандидоз или люба другая инфекц...,1
1,"Здравствуйте. Купался в реке, затянуло течение...",Преходящее нарушение питания мозга,1
2,"Добрый день, у меня были проблемы с носом всег...","Да,осложнения не исключены,вам стоит показатьс...",1
3,Здравствуйте! Писала недавно вопрос о задержке...,Здравствуйте! Да - ответ на оба вопроса.,1
4,"29,178,78 начал болеть гайморит , температура ...",Что вас сейчас беспокоит? Каковы ваши жалобы?,1
...,...,...,...
4804,здравствуйте. мне 16 лет. у меня эрозия. мне н...,"Вероятно это продолжительность лечения, 10 дне...",1
4805,Может ли болеть живот от молочного шоколада?по...,разумеется может. не советую экспериментировать,1
4806,Здравствуйте. Диагноз F 23.1 мне в последнее в...,Возможно это связано с вашим заболеванием.,1
4807,Здравствуйте! Скажите мне прописали уколы в/м ...,"Здравствуйте,они никак не действуют на сердце,...",1


Разобьем датасет на вопросы и ответы, тренировочную и тестовую выборки:

In [6]:
train_q, test_q, train_ans, test_ans = train_test_split(data.desc,
                                                        data.ans,
                                                        test_size=.2,
                                                        random_state=42)

In [7]:
device = torch.device("cuda")

Скачаем модель GPT от сбера:

In [8]:
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/551M [00:00<?, ?B/s]

Функции для простого препроцессинга (удаление непечатных знаков) и сборки датасета для обучения. При сборке датасета вопрос помечаем как '\nx:', а ответ - как '\ny:'

In [9]:
def preproc_text(text):
    res = str(text).strip()
    res = re.sub(r"\s+", " ", res)
    return res

def build_data(data_q, data_ans):
    data = []
    for idx, texts in enumerate(data_q):
        question = preproc_text(texts)
        answer = preproc_text(data_ans.iloc[idx])
        res = '\nx:' + question + '\ny:' + answer
        data.append(res)
    return data

In [10]:
train_data = build_data(train_q, train_ans)
test_data = build_data(test_q, test_ans)

In [22]:
tokenizer.pad_token = tokenizer.eos_token

Токенизируем тренировочный и тестовый датасеты:

In [65]:
train_encodings = tokenizer(train_data, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
test_encodings = tokenizer(test_data, padding='max_length', truncation=True, max_length=512, return_tensors='pt')

Класс для создания torch-датасета:

In [67]:
class QAMedicalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = QAMedicalDataset(train_encodings)
test_dataset = QAMedicalDataset(test_encodings)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Зададим аргуметы для дообучения модели:

In [69]:
training_args = TrainingArguments(
    output_dir="./qa_medical", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [70]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

Обучим модель:

In [71]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3792
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 2844
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Step,Training Loss
500,3.471800
1000,3.372600
1500,3.056900
2000,2.955800
2500,2.732700


Saving model checkpoint to ./qa_medical/checkpoint-800
Configuration saved in ./qa_medical/checkpoint-800/config.json
Model weights saved in ./qa_medical/checkpoint-800/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
Saving model checkpoint to ./qa_medical/checkpoint-1600
Configuration saved in ./qa_medical/checkpoint-1600/config.json
Model weights saved in ./qa_medical/checkpoint-1600/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
Saving model checkpoint to ./qa_medical/checkpoint-2400
Configuration saved in ./qa_medical/check

TrainOutput(global_step=2844, training_loss=3.073814016521899, metrics={'train_runtime': 1656.7046, 'train_samples_per_second': 6.867, 'train_steps_per_second': 1.717, 'total_flos': 2972458156032000.0, 'train_loss': 3.073814016521899, 'epoch': 3.0})

Сохраним модель на диск:

In [72]:
tokenizer.save_pretrained(path + 'tokenizer_qa_medical_gen')
model.save_pretrained(path + 'model_qa_medical_gen')

tokenizer config file saved in /content/drive/My Drive/GeekBrains/NLP/Course Project/tokenizer_qa_medical_gen/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/GeekBrains/NLP/Course Project/tokenizer_qa_medical_gen/special_tokens_map.json
Configuration saved in /content/drive/My Drive/GeekBrains/NLP/Course Project/model_qa_medical_gen/config.json
Model weights saved in /content/drive/My Drive/GeekBrains/NLP/Course Project/model_qa_medical_gen/pytorch_model.bin


In [74]:
tokenizer = AutoTokenizer.from_pretrained(path + 'tokenizer_qa_medical_gen')
model = AutoModelForCausalLM.from_pretrained(path + 'model_qa_medical_gen')

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file /content/drive/My Drive/GeekBrains/NLP/Course Project/model_qa_medical_gen/config.json
Model config GPT2Config {
  "_name_or_path": "/content/drive/My Drive/GeekBrains/NLP/Course Project/model_qa_medical_gen",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_

Функция для ведения диалога. Модель "держит в голове" контекст последних 10-ти сообщений.

In [75]:
def respond_to_dialog(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens, 
        eos_token_id=end_token_id,
        do_sample=True, 
        max_length=size+128, 
        repetition_penalty=3.2, 
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = re.findall(r'\ny:(.+)', decoded)[-1] 
    return result.strip()

Протестируем генератор:

In [76]:
seed = input('Начните диалог с ботом любой фразой\n')
history = [seed]
while True:
    result = respond_to_dialog(history[-10:])
    next_sentence = input(result + '\n')
    if next_sentence == 'Стоп':
        break
    history.append(result)
    history.append(next_sentence)

Начните диалог с ботом любой фразой
Здравствуйте, у меня болит голова, что мне делать?
Обратитесь к неврологу. Он определит причину болей и назначит лечение. Удачи!
А какие лекарства можно пить при мигренях?
Лечение в зависимости от причин мигрени может назначить невролог. Для начала обратитесь к психотерапевту. Если боль не проходит - покажитесь врачу повторно. Покажитесь ЛОР-врачу. Выложите фото жалоб. Возможно потребуется очная консультация врача-невролога. Удачи!
у меня болит живот. подскажите что делать
Выполните КТ органов брюшной полости с контрастированием или рентгенографию органов брюшной полости. С результатами исследований обратитесь к неврологу. Необходим осмотр невролога. Выполните УЗИ органов брюшной полости для исключения патологических изменений. Рекомендую обратиться к гастроэнтерологу. По результатам обследования врач назначит лечение. Удачи!
а какие лекарства можно принимать при болях в животе?
Для начала нужно выполнить МРТ органов брюшной полости,сдать анализ кров

**Вывод:**

С медицинской точки зрения, конечно, чушь. Но глупо было бы ожидать от генератора, обученного на медицинских диалогах, экспертные выводы по медицине. С точки зрения языка все выглядит очень красиво - ответы грамматически и логически связаны между собой, соответствуют медицинской тематике и даже улавливают симптоматику, а также похожи стилистически на ответы врача.